In [1]:
# import findspark
# spark_path = '../../2016UCSD/255/spark-1.6.0-bin-hadoop2.6/'
# findspark.init(spark_path)
# from pyspark import SparkContext
# sc = SparkContext(master="local[3]")


In [1]:
from get_features import *
import os, pickle
from sklearn import linear_model, datasets
from lib import *

In [2]:
def train_model(path, domain, model, emotion):
    train_file = 'Data/%s/trainData.json' % (domain)
    test_file = 'Data/%s/testData.json' % (domain)

    pickled_trainX = '%s/%s/%s/%s/trainX.pickle' % (path, domain, model, emotion)
    pickled_trainY = '%s/%s/%s/%s/trainY.pickle' % (path, domain, model, emotion)
    pickled_testX = '%s/%s/%s/%s/testX.pickle' % (path, domain, model, emotion)
    pickled_testY = '%s/%s/%s/%s/testY.pickle' % (path, domain, model, emotion)
    
    if not os.path.exists(pickled_trainX) or not os.path.exists(pickled_trainY):
        trainX, trainY = get_json_data(train_file, emotion)
        pickle.dump(trainX, open(pickled_trainX, 'w'))
        pickle.dump(trainY, open(pickled_trainY, 'w'))
    else:
        trainX = pickle.load(open(pickled_trainX, 'r'))
        trainY = pickle.load(open(pickled_trainY, 'r'))
    if not os.path.exists(pickled_testX) or not os.path.exists(pickled_testY):
        testX, testY = get_json_data(test_file, emotion)
        pickle.dump(testX, open(pickled_testX, 'w'))
        pickle.dump(testY, open(pickled_testY, 'w'))
    else:
        testX = pickle.load(open(pickled_testX, 'r'))
        testY = pickle.load(open(pickled_testY, 'r'))
    return trainX, trainY, testX, testY

In [26]:
path, domain, model, emotion = 'pickled' , 'Semeval_2007', 'model1', 'sad'
trainX, trainY, testX, testY = train_model(path, domain, model, emotion)

14182
14182


In [30]:
for i in range(7):
    c = 10 ** (-5 + i)
    logistic = model1(trainX, trainY, c = c)
    print 'c, train score, test score,'
    print c, evaluate(logistic, trainX, trainY), ' ', evaluate(logistic, testX, testY)

(250, 165) (250,)
c, train score, test score,
1e-05 0.616   0.709
(250, 165) (250,)
c, train score, test score,
0.0001 0.616   0.709
(250, 165) (250,)
c, train score, test score,
0.001 0.616   0.709
(250, 165) (250,)
c, train score, test score,
0.01 0.616   0.713
(250, 165) (250,)
c, train score, test score,
0.1 0.7   0.707
(250, 165) (250,)
c, train score, test score,
1 0.704   0.682
(250, 165) (250,)
c, train score, test score,
10 0.732   0.67


In [ ]:
# def getData(infile):
#     lines = [ line for line in open(infile, 'r').readlines()]
#     text, Y = [line.split(',')[0] for line in lines], \
#                         [int(line.split(',')[1]) for line in lines]
#     print len(text), len(Y)
#     assert(len(text) == len(Y), "we've got a problem")
#     wp = word_processor()
#     rdd = sc.parallelize(text).map(lambda x: wp.get_features1(x))
#     data = rdd.collect()
#     assert(len(data) == len(Y), "we've got a problem2")
#     return [data, Y]

In [ ]:
# logistic = train_model(path, domain)
# print 'train score, test score'
# print evaluate(logistic, trainX, trainY), ' ', evaluate(logistic, testX, testY)

In [ ]:
# logistic = train_model(trainX, trainY, c = 10)
# print 'train score, test score'
# print evaluate(logistic, trainX, trainY), ' ', evaluate(logistic, testX, testY)

## polarity with semeval-2017-task-5-subtask-1 data and TextBlob model

In [39]:
from textblob import TextBlob

In [37]:
trainFile = 'Data/polarity/trainData'
testFile = 'Data/polarity/testData'
trainData = [[ ''.join(line.split(',')[:-1]), float(line.strip('\n').split(',')[-1]) > 0] for line in open(trainFile, 'r').readlines()]
testData = [[ ''.join(line.split(',')[:-1]), float(line.strip('\n').split(',')[-1]) > 0] for line in open(testFile, 'r').readlines()]


In [38]:
print len(trainData), len(testData)

1500 578


In [51]:
def evaluate(data):
    pred = [TextBlob(d[0]).sentiment.polarity > 0 for d in data]
    Y = [d[1] for d in data]
    return np.sum(np.array(pred) == np.array(Y)) / float(len(data))
print evaluate(trainData), evaluate(testData)

 0.498 0.506920415225
